# 1. Téléchargement des bases

Données : https://www.data.gouv.fr/fr/datasets/repertoire-operationnel-des-metiers-et-des-emplois-rome/#_

Lien onisep : https://data.laregion.fr/explore/dataset/onisep-metiers-onisep/table/

Lien description ROME : 
https://candidat.pole-emploi.fr/marche-du-travail/fichemetierrome?codeRome=E1301

Liens entre ROME et autres bases (type PCS) : https://www.pole-emploi.org/opendata/repertoire-operationnel-des-meti.html?type=article



In [ ]:
import pandas as pd
import re

In [ ]:
!wget https://www.data.gouv.fr/fr/datasets/r/88342be1-06b8-4ab6-8ce9-83e117d21346  -O arborescence.xlsx

--2021-03-23 10:09:33--  https://www.data.gouv.fr/fr/datasets/r/88342be1-06b8-4ab6-8ce9-83e117d21346
Resolving www.data.gouv.fr (www.data.gouv.fr)... 37.59.183.93
Connecting to www.data.gouv.fr (www.data.gouv.fr)|37.59.183.93|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: http://pole-emploi.org/files/live/sites/peorg/files/documents/Statistiques-et-analyses/Open-data/ROME/ROME_ArboPrincipale.xlsx [following]
--2021-03-23 10:09:34--  http://pole-emploi.org/files/live/sites/peorg/files/documents/Statistiques-et-analyses/Open-data/ROME/ROME_ArboPrincipale.xlsx
Resolving pole-emploi.org (pole-emploi.org)... 185.215.64.44
Connecting to pole-emploi.org (pole-emploi.org)|185.215.64.44|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://pole-emploi.org/files/live/sites/peorg/files/documents/Statistiques-et-analyses/Open-data/ROME/ROME_ArboPrincipale.xlsx [following]
--2021-03-23 10:09:34--  https://pole-emploi.org/fi

In [ ]:
noms_col = ["Grands domaines","Domaines professionnels","ROME","Appelation","Code OGR"]
arborescence = pd.read_excel('arborescence.xlsx', sheet_name=1,
                             header = None, skiprows = 1,
                             names = noms_col)
arborescence[noms_col[1]] = arborescence[noms_col[0]]+arborescence[noms_col[1]]
arborescence[noms_col[2]] = arborescence[noms_col[1]]+arborescence[noms_col[2]]
for col in noms_col[:3]:
  arborescence[col] = arborescence[col].apply(lambda x: re.sub(" +", "", x))
arborescence

,Grands domaines,Domaines professionnels,ROME,Appelation,Code OGR
0,A,A,A,"Agriculture et Pêche, Espaces naturels et Espa...",
1,A,A11,A11,Engins agricoles et forestiers,
2,A,A11,A1101,Conduite d'engins agricoles et forestiers,
3,A,A11,A1101,Chauffeur / Chauffeuse de machines agricoles,11987
4,A,A11,A1101,Conducteur / Conductrice d'abatteuses,12862
...,...,...,...,...,...
11763,N,N44,N4403,Opérateur / Opératrice de branchement du résea...,17064
11764,N,N44,N4403,Opérateur / Opératrice de manoeuvre du réseau ...,17119
11765,N,N44,N4403,Opérateur / Opératrice de production du réseau...,17150
11766,N,N44,N4403,Opérateur / Opératrice fret du réseau ferré,17231


In [ ]:
liste_rome = arborescence["ROME"][arborescence["ROME"].apply(len)==5].unique()
print(len(liste_rome))
liste_rome

532


array(['A1101', 'A1201', 'A1202', 'A1203', 'A1204', 'A1205', 'A1301',
       'A1302', 'A1303', 'A1401', 'A1402', 'A1403', 'A1404', 'A1405',
       'A1406', 'A1407', 'A1408', 'A1409', 'A1410', 'A1411', 'A1412',
       'A1413', 'A1414', 'A1415', 'A1416', 'A1417', 'A1501', 'A1502',
       'A1503', 'A1504', 'B1101', 'B1201', 'B1301', 'B1302', 'B1303',
       'B1401', 'B1402', 'B1501', 'B1601', 'B1602', 'B1603', 'B1604',
       'B1701', 'B1801', 'B1802', 'B1803', 'B1804', 'B1805', 'B1806',
       'C1101', 'C1102', 'C1103', 'C1104', 'C1105', 'C1106', 'C1107',
       'C1108', 'C1109', 'C1110', 'C1201', 'C1202', 'C1203', 'C1204',
       'C1205', 'C1206', 'C1207', 'C1301', 'C1302', 'C1303', 'C1401',
       'C1501', 'C1502', 'C1503', 'C1504', 'D1101', 'D1102', 'D1103',
       'D1104', 'D1105', 'D1106', 'D1107', 'D1201', 'D1202', 'D1203',
       'D1204', 'D1205', 'D1206', 'D1207', 'D1208', 'D1209', 'D1210',
       'D1211', 'D1212', 'D1213', 'D1214', 'D1301', 'D1401', 'D1402',
       'D1403', 'D14

In [ ]:
arborescence["ROME"].apply(len)

0        1
1        3
2        5
3        5
4        5
        ..
11763    5
11764    5
11765    5
11766    5
11767    5
Name: ROME, Length: 11768, dtype: int64

In [ ]:
!wget https://api.emploi-store.fr/api/docs/romeopen/REF_ROME_CSV/1/RefRomeCsv.zip -O metadata.zip
!unzip metadata.zip
!rm metadata.zip

--2021-03-23 09:51:53--  https://api.emploi-store.fr/api/docs/romeopen/REF_ROME_CSV/1/RefRomeCsv.zip
Resolving api.emploi-store.fr (api.emploi-store.fr)... 185.215.64.15
Connecting to api.emploi-store.fr (api.emploi-store.fr)|185.215.64.15|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 1509706 (1.4M) [application/zip]
Saving to: ‘metadata.zip’

metadata.zip        100%[===================>]   1.44M   579KB/s    in 2.5s    

2021-03-23 09:51:56 (579 KB/s) - ‘metadata.zip’ saved [1509706/1509706]

Archive:  metadata.zip
  inflating: unix_coherence_item_v346_utf8.csv  
  inflating: unix_composition_bloc_v346_utf8.csv  
  inflating: unix_cr_gd_dp_appellations_v346_utf8.csv  
  inflating: unix_cr_gd_dp_v346_utf8.csv  
  inflating: unix_descriptif_rubrique_v346_utf8.csv  
  inflating: unix_domaine_professionnel_v346_utf8.csv  
  inflating: unix_grand_domaine_v346_utf8.csv  
  inflating: unix_item_arborescence_v346_utf8.csv  
  inflating: unix_item_v346_utf8.csv  
  i

In [ ]:
arborescence2 = pd.read_csv("unix_item_arborescence_v346_utf8.csv")
arbo3 = arborescence2[arborescence2["code_type_noeud"]==3]
arbo3

,code_ogr,code_type_referentiel,code_pere,code_noeud,libelle_item_arbo,code_item_arbo_associe,code_type_noeud,libelle_type_noeud,statut
125,21166,6,A11,A1101,Conduite d'engins agricoles et forestiers,6,3,FEUILLE,1
126,21167,6,A12,A1201,Bûcheronnage et élagage,7,3,FEUILLE,1
127,21168,6,A12,A1202,Entretien des espaces naturels,8,3,FEUILLE,1
128,21169,6,A12,A1203,Aménagement et entretien des espaces verts,9,3,FEUILLE,1
129,21170,6,A12,A1204,Protection du patrimoine naturel,10,3,FEUILLE,1
...,...,...,...,...,...,...,...,...,...
1179,17438,7,017,N4104,Courses et livraisons express,526,3,FEUILLE,1
1180,17337,7,013,N4104,Courses et livraisons express,526,3,FEUILLE,1
1181,17362,7,014,N4105,Conduite et livraison par tournées sur courte ...,527,3,FEUILLE,1
1182,17437,7,017,N4105,Conduite et livraison par tournées sur courte ...,527,3,FEUILLE,1


In [ ]:
arbo3[arbo3["code_noeud"] == "N4402"]

,code_ogr,code_type_referentiel,code_pere,code_noeud,libelle_item_arbo,code_item_arbo_associe,code_type_noeud,libelle_type_noeud,statut
655,21417,6,N44,N4402,Exploitation et manoeuvre des remontées mécani...,535,3,FEUILLE,1
1183,17343,7,013,N4402,Exploitation et manoeuvre des remontées mécani...,535,3,FEUILLE,1


In [ ]:
arborescence2[arborescence2["code_noeud"]=="013"]

,code_ogr,code_type_referentiel,code_pere,code_noeud,libelle_item_arbo,code_item_arbo_associe,code_type_noeud,libelle_type_noeud,statut
670,17035,7,Racine AR,013,"Métiers saisonniers, de vacances / Jobs d'été",0,2,NOEUD,1


In [ ]:
arborescence2[arborescence2["code_pere"]=="Racine AR"]["libelle_item_arbo"]

658    Métiers de l'environnement et du développement...
659    Métiers de la défense et de la sécurité publiq...
660    Métiers du patrimoine et de la restauration d'...
661                 Métiers de l'intelligence économique
662                              Métiers de la recherche
663    Métiers de la mer, du nautisme et de la constr...
664                            Métiers de l'aéronautique
665            Métiers de la sécurité publique et privée
666                                Métiers du multimédia
667    Métiers de l'humanitaire (hors santé et tertia...
668                                 Métiers du nucléaire
669                           Métiers auprès des enfants
670        Métiers saisonniers, de vacances / Jobs d'été
671        Métiers des services à la personne à domicile
672                  Métiers du sport et autour du sport
673                              Métiers de l'ingénierie
674    Métiers accessibles sans diplôme et sans expér...
Name: libelle_item_arbo, dtype:

In [ ]:
arborescence2[arborescence2["code_noeud"]=="N44"]

,code_ogr,code_type_referentiel,code_pere,code_noeud,libelle_item_arbo,code_item_arbo_associe,code_type_noeud,libelle_type_noeud,statut
124,21415,6,N,N44,Personnel sédentaire du transport ferroviaire ...,0,2,NOEUD,1


In [ ]:
rome2 = arborescence2[arborescence2["code_type_noeud"]==3]["code_noeud"]

In [ ]:
len(rome2.unique())

532

In [ ]:
liste_rome

['A1101',
 'A1201',
 'A1202',
 'A1203',
 'A1204',
 'A1205',
 'A1301',
 'A1302',
 'A1303',
 'A1401',
 'A1402',
 'A1403',
 'A1404',
 'A1405',
 'A1406',
 'A1407',
 'A1408',
 'A1409',
 'A1410',
 'A1411',
 'A1412',
 'A1413',
 'A1414',
 'A1415',
 'A1416',
 'A1417',
 'A1501',
 'A1502',
 'A1503',
 'A1504',
 'B1101',
 'B1201',
 'B1301',
 'B1302',
 'B1303',
 'B1401',
 'B1402',
 'B1501',
 'B1601',
 'B1602',
 'B1603',
 'B1604',
 'B1701',
 'B1801',
 'B1802',
 'B1803',
 'B1804',
 'B1805',
 'B1806',
 'C1101',
 'C1102',
 'C1103',
 'C1104',
 'C1105',
 'C1106',
 'C1107',
 'C1108',
 'C1109',
 'C1110',
 'C1201',
 'C1202',
 'C1203',
 'C1204',
 'C1205',
 'C1206',
 'C1207',
 'C1301',
 'C1302',
 'C1303',
 'C1401',
 'C1501',
 'C1502',
 'C1503',
 'C1504',
 'D1101',
 'D1102',
 'D1103',
 'D1104',
 'D1105',
 'D1106',
 'D1107',
 'D1201',
 'D1202',
 'D1203',
 'D1204',
 'D1205',
 'D1206',
 'D1207',
 'D1208',
 'D1209',
 'D1210',
 'D1211',
 'D1212',
 'D1213',
 'D1214',
 'D1301',
 'D1401',
 'D1402',
 'D1403',
 'D1404',


In [ ]:
len(liste_rome)
len(list(rome2))

1041

In [ ]:
rome2.unique()

AttributeError: ignored

In [ ]:
import numpy as np

In [ ]:
list(np.setdiff1d(list(rome2),liste_rome))

[]

# Test webscrapping

In [ ]:
import requests
from bs4 import BeautifulSoup

URL = 'https://www.onisep.fr/Ressources/Univers-Metier/Metiers/serigraphe'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')
results = soup.find(id="en-quoi-consiste-ce-metier")
print(results.prettify())
results.find_all('p')

<div class="ideo-block" id="en-quoi-consiste-ce-metier" tabindex="-1">
 <div class="ideo-inner-width">
  <h2 class="ideo-title-block" tabindex="-1">
   En quoi consiste ce métier ?
  </h2>
  <div class="ezxmltext-field">
   <p>
    Un dessin sur un T-shirt, un logo sur un porte-clefs, un motif qui orne un sac à dos ou une casquette, des autocollants... la sérigraphie est partout ! Cette technique d'impression traditionnelle permet au sérigraphe d'imprimer des motifs sur toutes sortes de supports : bois, tissu, verre, papier, métal... Les couleurs sont illimitées et le résultat obtenu s'utilise dans la publicité, la décoration, la mode, la céramique, l'aéronautique, les industries du verre, du luxe, de l'électronique, de l'automobile...
   </p>
   <p>
    Après avoir fabriqué une image négative du motif, le sérigraphe la reporte sur un écran en toile sous lequel est placé le support à imprimer. Il dépose l'encre sur le dessus de l'écran et la fait pénétrer avec une racle à travers les p

[<p>Un dessin sur un T-shirt, un logo sur un porte-clefs, un motif qui orne un sac à dos ou une casquette, des autocollants... la sérigraphie est partout ! Cette technique d'impression traditionnelle permet au sérigraphe d'imprimer des motifs sur toutes sortes de supports : bois, tissu, verre, papier, métal... Les couleurs sont illimitées et le résultat obtenu s'utilise dans la publicité, la décoration, la mode, la céramique, l'aéronautique, les industries du verre, du luxe, de l'électronique, de l'automobile...</p>,
 <p>Après avoir fabriqué une image négative du motif, le sérigraphe la reporte sur un écran en toile sous lequel est placé le support à imprimer. Il dépose l'encre sur le dessus de l'écran et la fait pénétrer avec une racle à travers les parties non obturées de l'écran. Le travail à la main se fait rare : l'informatique et les lignes d'impression numérisées notamment le supplantent peu à peu.</p>,
 <p>Créatif et minutieux, le sérigraphe doit également savoir s'adapter aux 

[<p>Un dessin sur un T-shirt, un logo sur un porte-clefs, un motif qui orne un sac à dos ou une casquette, des autocollants... la sérigraphie est partout ! Cette technique d'impression traditionnelle permet au sérigraphe d'imprimer des motifs sur toutes sortes de supports : bois, tissu, verre, papier, métal... Les couleurs sont illimitées et le résultat obtenu s'utilise dans la publicité, la décoration, la mode, la céramique, l'aéronautique, les industries du verre, du luxe, de l'électronique, de l'automobile...</p>,
 <p>Après avoir fabriqué une image négative du motif, le sérigraphe la reporte sur un écran en toile sous lequel est placé le support à imprimer. Il dépose l'encre sur le dessus de l'écran et la fait pénétrer avec une racle à travers les parties non obturées de l'écran. Le travail à la main se fait rare : l'informatique et les lignes d'impression numérisées notamment le supplantent peu à peu.</p>,
 <p>Créatif et minutieux, le sérigraphe doit également savoir s'adapter aux 

<div class="ideo-block" id="en-quoi-consiste-ce-metier" tabindex="-1">
 <div class="ideo-inner-width">
  <h2 class="ideo-title-block" tabindex="-1">
   En quoi consiste ce métier ?
  </h2>
  <div class="ezxmltext-field">
   <p>
    Un dessin sur un T-shirt, un logo sur un porte-clefs, un motif qui orne un sac à dos ou une casquette, des autocollants... la sérigraphie est partout ! Cette technique d'impression traditionnelle permet au sérigraphe d'imprimer des motifs sur toutes sortes de supports : bois, tissu, verre, papier, métal... Les couleurs sont illimitées et le résultat obtenu s'utilise dans la publicité, la décoration, la mode, la céramique, l'aéronautique, les industries du verre, du luxe, de l'électronique, de l'automobile...
   </p>
   <p>
    Après avoir fabriqué une image négative du motif, le sérigraphe la reporte sur un écran en toile sous lequel est placé le support à imprimer. Il dépose l'encre sur le dessus de l'écran et la fait pénétrer avec une racle à travers les p

In [ ]:
import requests
from bs4 import BeautifulSoup

URL = 'https://www.monster.com/jobs/search/?q=Software-Developer&where=Australia'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')
results = soup.find(id='ResultsContainer')
print(results.prettify())


<div class="mux-custom-scroll" data-extend="left" data-mux="customScroll" data-target="html" id="ResultsContainer">
 <div class="scrollable" id="ResultsScrollable">
  <script type="application/ld+json">
   {"@context":"https://schema.org","@type":"ItemList","mainEntityOfPage":{
            "@type":"CollectionPage","@id":"https://www.monster.com/jobs/search/?q=Software-Developer&amp;where=Australia"
            }
            ,"itemListElement":[

                 {"@type":"ListItem","position":1,"url":"https://job-openings.monster.com/software-engineer-full-stack-sydney-new-south-wales-sydney-nsw-us-mri-software/2abbd945-9020-4bc6-a0fe-bd16e0caa778"}
                    ,
                 {"@type":"ListItem","position":2,"url":""}
                    ,
                 {"@type":"ListItem","position":3,"url":"https://job-openings.monster.com/principal-consultant-automation-devops-melbourne-ga-us-dialog-group/15a6d27e-9b31-4417-9806-c13bf4d74e3f"}
                    ,
                 {"